# Домашнее задание: Pydantic


## Важно!

- При выполнении задания используем точные типы (`EmailStr`, `HttpUrl`, `SecretStr`, `Decimal`, конкретные `Enum`).
- Придерживаемся принципа разделения валидаций: проверка поля — в `field_validator`, сквозные зависимости — в `model_validator`


## Задача 1. Профиль пользователя (валидация полей)

Постройте модель профиля пользователя для внутренней CRM:

**Требования**
1. Обязательные поля: `id: UUID`, `email: EmailStr`, `name: str`.
2. Опциональные поля: `website: HttpUrl | None`, `bio: str | None`.
3. Пароль хранится как `SecretStr`, должен быть не короче 8 символов.
4. Имя (`name`) нормализуйте: тримминг + одна пробельная последовательность между словами + первая буква каждого слова заглавная.
5. Если указан `website`, домен сайта не должен совпадать с доменом `email` (смысл: личный сайт != корпоративная почта).

Подсказки: используйте `field_validator` для нормализации и локальных проверок; и `model_validator(mode="after")` для проверки зависимости `email` ↔ `website`.


In [1]:
!pip install -U pydantic[email,timezone] -q
!pip install -U pydantic_settings
!pip install -U sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Класс исключений. Реализован исключительно для удобства вывода типа возникающей ошибки.

In [2]:
class Validation_Error(Exception):
    def __init__(self, msg):
        self.message = msg

    def __str__(self):
        return self.message

In [3]:
from typing import Optional
from pydantic import BaseModel, Field, EmailStr, HttpUrl, SecretStr
from pydantic import field_validator, model_validator
from uuid import UUID, uuid4

class UserProfile(BaseModel):
    # Обязательные поля
    id: UUID = Field(default_factory=uuid4)
    email: EmailStr
    name: str
    password: SecretStr

    # Опциональные поля
    website: Optional[HttpUrl] = None
    bio: Optional[str] = None

    # TODO: нормализация имени
    @field_validator("name")
    @classmethod
    def normalize_name(cls, v: str) -> str:
        return ' '.join(v.strip().title().split())

    # TODO: проверка длины пароля
    @field_validator("password")
    @classmethod
    def password_strength(cls, v: SecretStr) -> SecretStr:
        if len(v.get_secret_value()) <= 7:
            raise Validation_Error("Пароль должен быть не короче 8 символов!")
        return v

    # TODO: сквозная проверка доменов email/website
    @model_validator(mode="after")
    def check_domains(self):
        if self.website is not None:
            email_domain = self.email.split('@')[1]
            website_domain = self.website.host
            if website_domain.startswith("www."):
                website_domain = website_domain[4:]

        if email_domain == website_domain:
            raise Validation_Error("Домен сайта не должен совпадать с доменом почты!")
                
        return self

    # печать информации о пользователе
    def __str__(self):
        data = {"id": self.id, "email": self.email, "name": self.name, 
                "pwd": self.password, "site": self.website, "bio": self.bio}
        return "\n".join(f"{key}: {value}" for key, value in data.items())

Попробуем проверить корректность работы на нескольких тестовых примерах:

In [4]:
print("Тест 1: Корректные данные")
try:
    user = UserProfile(id=uuid4(),
                       email="pupkin.vas@gmail.com",
                       name="Пупкин  Василий\t  ",
                       password=SecretStr("qwertyuiop"),
                       website="https://pupkin.com")
    print(user)
except ValidationError as e:
    print("Что-то пошло не так!")
    print(e)

Тест 1: Корректные данные
id: d0eb8482-2f79-4f16-b9d6-ff72f4cdc9ff
email: pupkin.vas@gmail.com
name: Пупкин Василий
pwd: **********
site: https://pupkin.com/
bio: None


In [6]:
print("Тест 2: Слишком короткий пароль")
try:
    user = UserProfile(id=uuid4(),
                       email="pupkin.vas@gmail.com",
                       name="Пупкин  Василий",
                       password=SecretStr("qwerty"),
                       website="https://pupkin.com")
    print(user)
except Validation_Error as e:
    print(e)

Тест 2: Слишком короткий пароль
Пароль должен быть не короче 8 символов!


In [7]:
print("Тест 3: Совпадение доменов")
try:
    user = UserProfile(id=uuid4(),
                       email="pupkin.vas@yandex.ru",
                       name="Пупкин  Василий",
                       password=SecretStr("qwertyuiop"),
                       website="https://www.yandex.ru")
    print(user)
except Validation_Error as e:
    print(e)

Тест 3: Совпадение доменов
Домен сайта не должен совпадать с доменом почты!


## Задача 2. Валидация функции заказа (`@validate_call`)

Реализуйте функцию `place_order`, которая принимает:
- `user_id: UUID`
- `sku: str` (артикул, только заглавные буквы/цифры, длина 3–12)
- `quantity: int` (>0)
- `price: Decimal` (>= 0), округляется банковским методом до 2 знаков

Функция должна возвращать словарь с ключами: `user_id`, `sku`, `quantity`, `price`, `amount` (quantity × price).

Используйте `@validate_call` и локальные проверки через обычный код (или вспомогательные валидаторы `TypeAdapter` не используем).


In [8]:
from pydantic import validate_call, Field
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
import re

# TODO: реализуйте функцию с @validate_call
@validate_call
def place_order(user_id: UUID, 
                sku: str = Field(..., min_length=3, max_length=12),
                quantity: int = Field(..., gt=0), 
                price: Decimal = Field(..., ge=0)):
    if not sku.isalnum():
        raise Validation_Error("Артикул должен содержать только буквы и цифры!")
    if not sku.isupper():
        raise Validation_Error("В артикуле могут быть только заглавные буквы!")

    price = price.quantize(Decimal('0.01'), rounding=ROUND_HALF_EVEN)
    
    return {"user_id": user_id,
           "sku": sku,
           "quantity": quantity,
           "price": price,
           "amount": quantity * price}

Проверка работы функции

In [9]:
print("Тест 1: Корректные данные + проверка округления")
try:
    for number in "19.999 18.999".split():
        order = place_order(user_id=uuid4(),
                            sku="Q1W2E3R4T5Y6",
                            quantity=5,
                            price=Decimal(number)
                           )
        for key, value in order.items():
            print(f"{key}: {value} ({type(value)})")
        print()
except Exception as e:
    print("Что-то пошло не так!")
    print(e)

Тест 1: Корректные данные + проверка округления
user_id: 9b99bf7f-5834-4d05-83ff-50ca205b31ea (<class 'uuid.UUID'>)
sku: Q1W2E3R4T5Y6 (<class 'str'>)
quantity: 5 (<class 'int'>)
price: 20.00 (<class 'decimal.Decimal'>)
amount: 100.00 (<class 'decimal.Decimal'>)

user_id: 4bfb2c54-7d78-4790-9c9f-3665e7a9db3c (<class 'uuid.UUID'>)
sku: Q1W2E3R4T5Y6 (<class 'str'>)
quantity: 5 (<class 'int'>)
price: 19.00 (<class 'decimal.Decimal'>)
amount: 95.00 (<class 'decimal.Decimal'>)



In [10]:
print("Тест 2: некорректный артикул")
for curr_sku in "QW qwerty123 qwerty!".split():
    try:
        order = place_order(user_id=uuid4(),
                    sku=curr_sku,
                    quantity=5,
                    price=Decimal('10')
                   )
        print(order)
    except Exception as e:
        print(e)

Тест 2: некорректный артикул
1 validation error for place_order
sku
  String should have at least 3 characters [type=string_too_short, input_value='QW', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short
В артикуле могут быть только заглавные буквы!
Артикул должен содержать только буквы и цифры!


In [11]:
print("Тест 3: некорректные числа")
try:
    order = place_order(user_id=uuid4(),
                sku="Q1W2E3R4T5Y6",
                quantity=0,  # неверное количество
                price=Decimal('10')
               )
    print(order)
except Exception as e:
    print(e)
    print()
try:
    order = place_order(user_id=uuid4(),
                sku="Q1W2E3R4T5Y6",
                quantity=5,
                price=Decimal('-3')  # неверная цена
               )
    print(order)
except Exception as e:
    print(e)

Тест 3: некорректные числа
1 validation error for place_order
quantity
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

1 validation error for place_order
price
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=Decimal('-3'), input_type=Decimal]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal


## Задача 3. Модель заказа с бизнес-правилами

Смоделируйте заказ в магазине цифровых товаров.

**Требования**
- `OrderStatus: Enum` со значениями `new`, `paid`, `delivered`, `canceled`.
- Модель `OrderItem`:
  - `sku: str` как в задаче 2
  - `qty: int` (>0)
  - `unit_price: Decimal` (>=0) округление до 2 знаков
- Модель `Order`:
  - `id: UUID`
  - `user_email: EmailStr`
  - `items: list[OrderItem]` (не пустой)
  - `status: OrderStatus = 'new'`
  - `created_at: datetime` (по умолчанию `datetime.utcnow`)
  - Расчитанное поле `total: Decimal` — сумма по всем позициям
  - В `model_validator(mode="after")` запретите переход в `paid`/`delivered` при `total == 0` и запретите пустые корзины.

**Важно:** используйте только инструменты `pydantic` и стандартную библиотеку.


In [12]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
from datetime import datetime
from enum import Enum

class OrderStatus(str, Enum):
    # TODO: перечислите статусы
    NEW = 'new'
    PAID = 'paid'
    DELIVERED = 'delivered'
    CANCELED = 'canceled'

class OrderItem(BaseModel):
    # TODO: опишите поля
    # хотелось тут через Field сорганизовать ограничения, но в условии задачи это запрещено
    sku: str
    qty: int
    unit_price: Decimal

    @field_validator("sku")
    @classmethod
    def sku_format(cls, v: str) -> str:
        if not (3 <= len(v) <= 12):
            raise Validation_Error("Артикул должен содержать от 3 до 12 символов!")
        if not v.isalnum():
            raise Validation_Error("Артикул должен содержать только буквы и цифры!")
        if not v.isupper():
            raise Validation_Error("В артикуле могут быть только заглавные буквы!")
        return v

    @field_validator("qty")
    @classmethod
    def qty_positive(cls, v: int) -> int:
        if v <= 0:
            raise Validation_Error(f"Количество не может быть отрицательным: {v}!")
        return v

    @field_validator("unit_price")
    @classmethod
    def price_non_negative(cls, v: Decimal) -> Decimal:
        if v <= 0:
            raise ValidationError(f"Цена не может быть отрицательной: {v}!")
        return v.quantize(Decimal('0.01'), rounding=ROUND_HALF_EVEN)

    def __str__(self):
        return f"Заказ: {self.sku}; количество: {self.qty}, стоимость: {self.unit_price}"

    # нужен для корректного отображения элемента класса в списке
    def __repr__(self):
        return f"Товар: {self.sku}; количество: {self.qty}, стоимость: {self.unit_price}"


class Order(BaseModel):
    # TODO: опишите поля
    id: UUID = Field(default_factory=uuid4)
    user_email: EmailStr
    items: List[OrderItem]
    status: OrderStatus = OrderStatus.NEW
    created_at: datetime = Field(default_factory=datetime.utcnow)

    @property
    def total(self) -> Decimal:
        # сумма по всем позициям
        return sum(item.unit_price * item.qty for item in self.items)
    
    @model_validator(mode="after")
    def check_business_rules(self):
        if len(self.items) == 0:
            raise Validation_Error("Корзина не должна быть пустой!")
        if self.status in [OrderStatus.PAID, OrderStatus.DELIVERED] and self.total == 0:
            msg = "Статус заказа не может быть ОПЛАЧЕН/ДОСТАВЛЕН при общей сумме заказа, равной нулю!"
            raise Validation_Error(msg)
        return self

    def __str__(self):
        result = f"Заказ: {self.id}; почта: {self.user_email}\n"
        result += f"Статус заказа: {self.status}; создан: {self.created_at}\n"
        result += f"Состав заказа:\n{'\n'.join(str(x) for x in self.items)}"
        return result  

Пример тестирования функционала.  
Охватил не все случаи, а только часть. В реальной программе написал бы покрытие тестами вообще всех возможных вариантов.

In [13]:
print("Тест 1: Корректный заказ")
try:
    order = Order(
        user_email="pupkin.vas@gmail.com",
        items=[
            OrderItem(sku="ITEM01", qty=2, unit_price=Decimal("9.99")),
            OrderItem(sku="ITEM02", qty=1, unit_price=Decimal("8.99"))
        ]
    )
    print(order)
except Exception as e:
    print("Что-то пошло не так!")
    print(f"Ошибка: {e}")

Тест 1: Корректный заказ
Заказ: 35ffbd4d-eb3a-4f90-86d3-8184e116c3e5; почта: pupkin.vas@gmail.com
Статус заказа: OrderStatus.NEW; создан: 2025-11-23 20:26:53.864573
Состав заказа:
Заказ: ITEM01; количество: 2, стоимость: 9.99
Заказ: ITEM02; количество: 1, стоимость: 8.99


In [14]:
print("Тест 2: Пустая корзина")
try:
    order = Order(
        user_email="pupkin.vas@gmail.com",
        items=[]
    )
    print(order)
except Exception as e:
    print(e)

Тест 2: Пустая корзина
Корзина не должна быть пустой!


In [15]:
print("Тест 3: Ошибка в одном из товаров")
try:
    order = Order(
        user_email="pupkin.vas@gmail.com",
        items=[
            OrderItem(sku="ITEM01", qty=2, unit_price=Decimal("-10")),
            OrderItem(sku="ITEM02", qty=1, unit_price=Decimal("8.99"))
        ]
    )
    print(order)
except Exception as e:
    print(e)

Тест 3: Ошибка в одном из товаров
name 'ValidationError' is not defined


## Задача 4. Конфигурация приложения (`BaseSettings`)

Опишите настройки подключения к внешнему API:

- `APISettings(BaseSettings)` с полями:
  - `base_url: HttpUrl`
  - `token: SecretStr`
  - `timeout_sec: int = 5` (1–60)
  - `retries: int = 2` (0–10)
- Используйте `model_config = ConfigDict(env_prefix="API_", env_file=".env", extra="ignore")`
- Проверьте, что значения корректно читаются из переменных окружения.

В тесте ниже среда заполняется вручную.


In [16]:
import os
from pydantic_settings import BaseSettings
from pydantic import ConfigDict, SecretStr, HttpUrl, field_validator, Field, ValidationError

class APISettings(BaseSettings):
    # TODO: поля и валидации
    base_url: HttpUrl
    token: SecretStr
    timeout_sec: int = Field(default=5, ge=1, le=60)
    retries: int = Field(default=2, ge=0, le=10)

    # пример проверки диапазона для timeout_sec / retries
    @field_validator("timeout_sec", "retries")
    @classmethod
    def check_ranges(cls, v: int, info):
        # валидация осуществлена через Field
        return v

    model_config = ConfigDict(
        # TODO: настройте env_prefix и прочие опции
        env_prefix="API_",
        env_file=".env",
        extra="ignore",
    )

    def __str__(self):
        result = f"base_url: {self.base_url}\n"
        result += f"token: {self.token}\n"
        result += f"timeout_sec: {self.timeout_sec}\n"
        result += f"retries: {self.retries}"
        return result

Проверка чтения значений из переменных окружения

In [17]:
# Устанавливаем переменные окружения
os.environ["API_BASE_URL"] = "https://api.example.com/"
os.environ["API_TOKEN"] = "token123"
os.environ["API_TIMEOUT_SEC"] = "10"
os.environ["API_RETRIES"] = "3"
try:
    settings = APISettings()
    print(settings)
except ValidationError as e:
    print(e)

base_url: https://api.example.com/
token: **********
timeout_sec: 10
retries: 3


## Задача 5. Извлечение из ORM (`from_attributes=True`)

Создайте простую SQLAlchemy-модель `SAUser(id, email, is_active)` (in-memory, без БД) и соответствующую модель Pydantic:

- Pydantic-модель `UserOut` с полями `id: UUID`, `email: EmailStr`, `is_active: bool`.
- Включите поддержку `from_attributes` в `model_config`.
- Создайте инстанс `SAUser` и провалидируйте его через `UserOut.model_validate(sa_user_instance)`.

Проверьте, что преобразование сработало.


In [18]:
from typing import Optional
from sqlalchemy import Column, String, Boolean
from sqlalchemy.orm import declarative_base
from uuid import uuid4
from pydantic import BaseModel, EmailStr, ConfigDict

Base = declarative_base()

class SAUser(Base):
    __tablename__ = "users"
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    email = Column(String, nullable=False)
    is_active = Column(Boolean, default=True)

    def __init__(self, email: str, is_active: bool = True):
        self.id = str(uuid4())
        self.email = email
        self.is_active = is_active

    def __str__(self):
        result = f"SAUser: {self.id}\n"
        result += f"email: {self.email}\n"
        result += f"is_active: {self.is_active}"
        return result

class UserOut(BaseModel):
    # TODO: опишите поля и включите from_attributes
    id: UUID
    email: EmailStr
    is_active: bool

    model_config = ConfigDict(
        # TODO: включите режим атрибутов
        from_attributes=True
    )

    def __str__(self):
        result = f"UserOut: {self.id}\n"
        result += f"email: {self.email}\n"
        result += f"is_active: {self.is_active}"
        return result

Тестирование

In [19]:
sa_user = SAUser(email="  pupkin.vas@gmail.com  ", is_active=True)
print(f"id: {sa_user.id}, тип id: {type(sa_user.id)}")
print(f"email: {sa_user.email}, тип email: {type(sa_user.email)}")
print()

# валидация через Pydantic:
try:
    user_out = UserOut.model_validate(sa_user)
    print(f"id: {user_out.id}, тип id: {type(user_out.id)}")
    print(f"email: {user_out.email}, тип email: {type(user_out.email)}")
except Exception as e:
    print(e)

id: de733f45-34ba-497a-9c7a-8cbab13038d9, тип id: <class 'str'>
email:   pupkin.vas@gmail.com  , тип email: <class 'str'>

id: de733f45-34ba-497a-9c7a-8cbab13038d9, тип id: <class 'uuid.UUID'>
email: pupkin.vas@gmail.com, тип email: <class 'str'>


## Задача 6. JSON Schema и дружелюбные ошибки

1. Для модели из задачи 3 сгенерируйте JSON Schema (метод `model_json_schema`) и запишите его в переменную `ORDER_SCHEMA`.
2. Реализуйте функцию `safe_create_order(data: dict) -> tuple[bool, str]`, которая:
   - пытается создать `Order` из входного `dict`,
   - при успехе возвращает `(True, "<total=...>")`,
   - при ошибке возвращает `(False, "<короткое сообщение об ошибке>")` без стек-трейса.

Не используйте сторонние библиотеки.

In [33]:
import json

# Используем модели из задачи 3: OrderStatus, OrderItem, Order
ORDER_SCHEMA = Order.model_json_schema()

def safe_create_order(data: dict) -> tuple[bool, str]:
    # TODO: реализуйте безопасное создание заказа
    try:
        order = Order(**data)
        return True, order
    except ValidationError as e:
        # return False, e.errors()
        errors = []
        for error in e.errors():
            field = " -> ".join(str(loc) for loc in error['loc'])
            msg = error['msg']
            
            # Делаем сообщения более понятными
            if "string_too_short" in msg:
                errors.append(f"Поле '{field}' слишком короткое")
            elif "string_too_long" in msg:
                errors.append(f"Поле '{field}' слишком длинное")
            elif "greater_than" in msg:
                errors.append(f"Поле '{field}' должно быть больше 0")
            elif "less_than" in msg:
                errors.append(f"Поле '{field}' должно быть меньше")
            elif "url" in msg:
                errors.append(f"Поле '{field}' должно быть валидным URL")
            elif "email" in msg:
                errors.append(f"Поле '{field}' должно быть валидным email")
            elif "@-sign" in msg:
                errors.append(f"адрес почты'{field}' должен содержать @")
            else:
                errors.append(f"Ошибка в поле '{field}': {msg}")
        return False, errors
    except Exception as e:
        return False, e

Тестирование

In [28]:
print("Тест 1: Корректное использование")
data = {"user_email": "pupkin.vasgmail.com",
        "items": [
            {"sku": "ITEM01", "qty": 2, "unit_price": "25"},
            {"sku": "ITEM02", "qty": 1, "unit_price": "100"}
        ]
    }
flag, result = safe_create_order(data)
if flag:
    print("Успешная обработка заказа")
    print(order)
else:
    print("Обработка заказа прошла с ошибками:")
    print(result)

Тест 1: Корректное использование
Успешная обработка заказа
Заказ: 35ffbd4d-eb3a-4f90-86d3-8184e116c3e5; почта: pupkin.vas@gmail.com
Статус заказа: OrderStatus.NEW; создан: 2025-11-23 20:26:53.864573
Состав заказа:
Заказ: ITEM01; количество: 2, стоимость: 9.99
Заказ: ITEM02; количество: 1, стоимость: 8.99


In [34]:
print("Тест 2: Заказ с максимальным число ошибок")
data = {"user_email": "pupkin.vas.gmail.com",
        "items": [
            {"sku": "ITEM01", "qty": 2.3, "unit_price": "10"},
            {"sku": "ITEM02", "qty": 1, "unit_price": "100"}
        ]
    }
flag, result = safe_create_order(data)
if flag:
    print("Успешная обработка заказа")
    print(order)
else:
    print("Обработка заказа прошла с ошибками:")
    print(*result, sep="\n")

Тест 2: Заказ с максимальным число ошибок
Обработка заказа прошла с ошибками:
Поле 'user_email' должно быть валидным email
Ошибка в поле 'items -> 0 -> qty': Input should be a valid integer, got a number with a fractional part
